In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb, parambokeh
import pandas as pd
import dask.dataframe as dd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, PlotSize

hv.extension('bokeh')

ddf = dd.read_parquet('./data/nyc_taxi.parq/').persist()

# Dashboards Visualizing Hundreds of Millions of Datapoints in 30 Lines of Python

### Philipp Rudiger, James A. Bednar and Jean-Luc Stevens

* Jupyter Dashboards is the only way to generate a graphical dashboard layout by interactively positioning them on a grid
* Paramnb is the only param UI extension that has been released and has support for most parameter types but can only be deployed using the obsolete Jupyter Dashboards
* ParamBokeh is pre-release and allows usage both in the notebook and deployment on bokeh server. It can already be used in the notebook and on bokeh server, however in the notebook it cannot easily be used to completely rerender a plot.

In [ ]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/'
                                   'World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))
tile_options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)  

passenger_counts = sorted(ddf.passenger_count.unique().compute().tolist())

class Options(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.ObjectSelector(default=1,          objects=passenger_counts)

    def make_view(self, x_range, y_range, alpha, colormap, plot, passengers):
        map_tiles = tiles(style=dict(alpha=alpha), plot=tile_options) 

        df_filt = ddf[ddf.passenger_count==passengers]
        points = hv.Points(df_filt, kdims=[plot+'_x', plot+'_y'])
        taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=colormap,
                               dynamic=False, x_range=x_range, y_range=y_range)
        return map_tiles * taxi_trips
    
    def view(self):
        return hv.DynamicMap(self.make_view, streams=[self, RangeXY()])

selector = Options(name="")
paramnb.Widgets(selector, callback=selector.event)
selector.view()